
# 📊 Compare All Models — Automatic Accuracy Extraction

This notebook automatically compares all trained models (TensorFlow + PyTorch).
It extracts the **best validation accuracy** from `logs/` and produces a summary table and bar chart.

✅ **Supported Models:**
- EfficientNetB0 (TensorFlow)
- ResNet50 (PyTorch)
- Vision Transformer (ViT, PyTorch)
- MobileNetV3-Small (PyTorch)

**Outputs:**
```
models/comparison_summary.csv
models/comparison_bar_chart.png
```


In [ ]:

import os, re
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path.cwd()
LOGS = ROOT / "logs"
MODELS = ROOT / "models"
MODELS.mkdir(exist_ok=True, parents=True)

print("📁 Logs dir:", LOGS)
print("📁 Models dir:", MODELS)


In [ ]:

def extract_best_accuracy(log_text):
    # Extracts best Val accuracy from log text
    accuracies = []
    for line in log_text.splitlines():
        # Match patterns like "Val Accuracy: 87.5" or "Val=87.5%"
        match = re.search(r"Val[ _]?Acc(?:uracy)?:?\s*([0-9]+\.?[0-9]*)", line, re.I)
        if match:
            accuracies.append(float(match.group(1)))
        else:
            alt = re.search(r"Val=([0-9]+\.?[0-9]*)%", line)
            if alt:
                accuracies.append(float(alt.group(1)))
    return max(accuracies) if accuracies else None


In [ ]:

data = []

entries = [
    ("EfficientNetB0", "TensorFlow", "tf_efficientnet_best.keras"),
    ("ResNet50", "PyTorch", "resnet50_best.pth"),
    ("Vision Transformer", "PyTorch", "vit_best.pth"),
    ("MobileNetV3-Small", "PyTorch", "mobilenetv3_best.pth"),
]

for name, framework, model_file in entries:
    model_path = MODELS / model_file
    found = model_path.exists()
    accuracy = None
    epochs = None

    if LOGS.exists():
        for log_file in LOGS.glob("*.log"):
            if name.split()[0].lower() in log_file.name.lower():
                text = log_file.read_text(errors="ignore")
                acc = extract_best_accuracy(text)
                if acc:
                    accuracy = acc
                epoch_matches = re.findall(r"Epoch\s*(\d+)", text)
                if epoch_matches:
                    epochs = int(max(epoch_matches))
                break

    data.append({
        "Model": name,
        "Framework": framework,
        "File Exists": "✅" if found else "❌",
        "Best Accuracy (%)": accuracy if accuracy else "N/A",
        "Epochs": epochs if epochs else "N/A",
    })

df = pd.DataFrame(data)
df


In [ ]:

plt.figure(figsize=(8,5))
valid_df = df[df["Best Accuracy (%)"] != "N/A"]
if not valid_df.empty:
    plt.bar(valid_df["Model"], valid_df["Best Accuracy (%)"], color="cornflowerblue")
    plt.ylabel("Accuracy (%)")
    plt.title("Model Validation Accuracy Comparison")
    plt.xticks(rotation=20, ha="right")
    plt.tight_layout()
    chart_path = MODELS / "comparison_bar_chart.png"
    plt.savefig(chart_path)
    print("📊 Saved chart →", chart_path)
    plt.show()
else:
    print("⚠️ No valid accuracy values found in logs.")


In [ ]:

csv_path = MODELS / "comparison_summary.csv"
df.to_csv(csv_path, index=False)
print("💾 Summary saved to:", csv_path)

if not df.empty and (df["Best Accuracy (%)"] != "N/A").any():
    df_num = df[df["Best Accuracy (%)"] != "N/A"].copy()
    df_num["Best Accuracy (%)"] = df_num["Best Accuracy (%)"].astype(float)
    best = df_num.loc[df_num["Best Accuracy (%)"].idxmax()]
    print(f"🏆 Best Model: {best['Model']} ({best['Best Accuracy (%)']}%) using {best['Framework']}")
else:
    print("⚠️ No valid models or accuracy results found.")
